<a href="https://colab.research.google.com/github/bob8dod/NLP_project-Review_Analysis/blob/main/Crawling/11st_(ver.3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 크롤링(11번가)

In [4]:
import json
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
import time
from multiprocessing import Pool, Manager


headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
products = 2211689236

url_basic = 'https://www.11st.co.kr/products/{}'.format(products)
data = requests.get(url_basic, headers=headers)
soup = BeautifulSoup(data.text, 'html.parser')

category_path = soup.find('div', attrs={'class':'c_product_category_path'}).find_all('em', attrs={'class':'selected'})
categories = set()
for cate in category_path:
    categories.add(cate.text)
product_name = soup.find('h1', attrs={'class':'title'}).text.strip()
img_src = soup.find('div', attrs={'class':'img_full'}).find('img')['src']
price = soup.find('ul', attrs={'class':'price_wrap'}).find('span',attrs={'class':'value'}).text
print(product_name)
print(categories, price)
print(img_src)
# except: # For Amazon
#     name = soup.find('meta',attrs={'property':'og:title'})['content']
#     cate_price = soup.find('meta',attrs={'property':'og:description'})['content'].strip()
#     img = soup.find('meta',attrs={'property':'og:image'})['content']
#     print(name)
#     print(cate_price)
#     print(img)

manager = Manager()
text = manager.list()
score = manager.list()
def Crawling(pageNo):
    num = 0
    
    try:
        url = 'https://m.11st.co.kr/products/v1/app/products/{}/reviews/list?pageNo={}&sortType=01&pntVals=&rtype=&themeNm='.format(products, pageNo)
        response = urlopen(url)
        json_data = json.load(response)
        for i in range(len(json_data['review']['list'])):
            if json_data['review']['list'][i]['subject']:
                num += 1
                text.append(''.join([str(num),'|', json_data['review']['list'][i]['subject']]))
                score.append(json_data['review']['list'][i]['evlPnt'])
    except:
        print('리뷰가 끝났습니다. 총 개수:', num)

if __name__ == '__main__':
    start_time = time.time()
    pool = Pool(processes=8) #4개의 프로세스 동시에 작동
    pool.map(Crawling,range(1,50)) #title_to_list라는 함수에 1 ~ end까지 10씩늘려가며 인자로 적용
    pool.close()
    pool.join()
    print(len(text))
    print("실행 시간 : %s초" % (time.time() - start_time))

프레시지 소고기 샤브샤브 (2인분) 밀키트 쿠킹박스
{'식품'} 12,520
https://cdn.011st.com/11dims/resize/600x600/quality/75/11src/pd/21/6/8/9/2/3/6/OmXUE/2211689236_B.jpg
490
실행 시간 : 7.451384544372559초


## New Text2Csv


In [5]:
import pandas as pd
line = [[r.split('|')[0],r.split('|')[1]] for r in text if len(r.split('|'))>1]
data = pd.DataFrame(line, columns=['idx','review'])
print(len(data))
data.head(3)

490


,idx,review
0,1,자주 구매해먹는 샤브샤브 재료도 좋고 맛도 있어요!!
1,2,저번에도 시켰는데 맛있어서 또 구매했어요
2,3,지난번에 사봤는데 맛이 괜찮아서 다시 주문합니다.
